# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,23.20,72,36,8.03,YE,1735862822
1,1,port-aux-francais,-49.3500,70.2167,5.60,99,100,5.37,TF,1735862823
2,2,grytviken,-54.2811,-36.5092,10.93,79,14,3.26,GS,1735862825
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.70,72,52,6.79,SH,1735862826
4,4,east london,-33.0153,27.9116,21.87,87,55,4.21,ZA,1735862827


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.scatter(
    x='Lng', 
    y='Lat', 
    color='Humidity', 
    size='Humidity', 
    hover_cols=['City', 'Country', 'Humidity'],
    title="City Locations with Humidity Size"
)
# Display the map
city_map


:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Humidity'] >= 60) & (city_data_df['Humidity'] <= 80)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,23.20,72,36,8.03,YE,1735862822
2,2,grytviken,-54.2811,-36.5092,10.93,79,14,3.26,GS,1735862825
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.70,72,52,6.79,SH,1735862826
5,5,iqaluit,63.7506,-68.5145,-5.15,74,75,3.60,CA,1735862535
8,8,adamstown,-25.0660,-130.1015,24.54,76,100,5.79,PN,1735862831


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hadibu,YE,12.6500,54.0333,72,
2,grytviken,GS,-54.2811,-36.5092,79,
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,72,
5,iqaluit,CA,63.7506,-68.5145,74,
8,adamstown,PN,-25.0660,-130.1015,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # 10 km radius for hotel search
params = {
    'apiKey': geoapify_key,
    'categories': 'tourism.hotel',  # Search for hotels
    'limit': 1  # Get the first result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"point:{lat},{lng}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hadibu - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
smithers - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
afaahiti - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
gizo - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
morondava - nearest hotel: No hotel found
fort mcmurray - nearest hotel: No hotel found
port blair - nearest hotel: No hotel found
alofi - nearest hotel: No hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hadibu,YE,12.6500,54.0333,72,No hotel found
2,grytviken,GS,-54.2811,-36.5092,79,No hotel found
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,72,No hotel found
5,iqaluit,CA,63.7506,-68.5145,74,No hotel found
8,adamstown,PN,-25.0660,-130.1015,76,No hotel found
...,...,...,...,...,...,...
567,kas,TR,36.2018,29.6377,75,No hotel found
573,sabang,ID,5.8933,95.3214,76,No hotel found
574,cockburn town,TC,21.4612,-71.1419,75,No hotel found
577,cottage grove,US,44.8277,-92.9438,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
city_map_with_hotel = hotel_df.hvplot.scatter(
    x='Lng', 
    y='Lat', 
    color='Humidity', 
    size='Humidity', 
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    title="City Locations with Hotels and Humidity"
)

# Display the map
city_map_with_hotel


:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)